In [ ]:
from main import *

m = treeSolution("19104960716", "Ldb20040716?")

In [5]:
import json

def to_json(question, answer, title=""):

    f = open(f"data/{title}.json", "+w", encoding="utf-8")
    json.dump(dict(zip(question, answer)), f,
              indent=4, ensure_ascii=False)
    f.close()

In [33]:
import re
import time

def getQuestionAnswer(driver, link, save=False, title=''):
    
    driver.get(link)
    time.sleep(1)
    question = []
    answer = []
    try:
        if len(driver.find_elements(By.TAG_NAME, "font")[0].text.split("\n")) <= 1 :
            fu_text = driver.find_elements(By.TAG_NAME, "font")[1].text.split("\n")
        else: fu_text = driver.find_elements(By.TAG_NAME, "font")[0].text.split("\n")
        fu_text = [x for x in fu_text if x != '']
        flag = True
        for index in range(len(fu_text)):
            ti = fu_text[index]
            if "第一章" not in  ti and flag: 
                continue
            else: flag = False
            if fu_text[index][0].isdigit(): 
                tmp = index
                question.append(fu_text[index])
            if fu_text[index][:4] in ["我的答案"] or fu_text[index][:2] in ["答案"]:
                length = index - tmp - 1
                true_answer = re.sub("[:]", "：",fu_text[index])
                true_answer = true_answer.split("：")
                true_answer = [x.strip(" 【】") for x in true_answer]
                if len(true_answer) == 1: 
                    true_answer = fu_text[index][-1]
                else:
                    true_answer = true_answer[-1]
                answer_dic = {(chr(65+length-v)):v for v in range(length, 0, -1)}
                if len(true_answer) == 1:
                    if true_answer in ["X", "错", 'x', '×']: answer.append(0)
                    elif true_answer in ["√", "对"]: answer.append(1)
                    else:
                        option = answer_dic[true_answer]
                        answer.append(fu_text[index - option])
                else:
                    options = []
                    for option in true_answer:
                        option = answer_dic[option]
                        options.append(fu_text[index - option])
                    answer.append(options)
    except Exception as e:
        print(true_answer)
        print(e)
        # return question, answer

    return question, answer

In [72]:
import time

def getQuestionAnswer2(driver, link, save=False, title=''):
    
    driver.get(link)
    time.sleep(1)
    question = []
    answers = []
    flag = True
    q_flag = False
    try:
        if len(driver.find_elements(By.TAG_NAME, "font")[0].text.split("\n")) <= 1 :
            fu_text = driver.find_elements(By.TAG_NAME, "font")[1].text.split("\n")
        else: fu_text = driver.find_elements(By.TAG_NAME, "font")[0].text.split("\n")
        fu_text = [x for x in fu_text if x != '']
        for index in range(len(fu_text)):
            ti = fu_text[index]
            if "第一章" not in ti and flag:  # 开始记录章节答题flag
                continue
            else: flag = False
            if fu_text[index].split("、")[0].isdigit(): 
                q1 = index
                q_flag = True
            if fu_text[index][0] == "A" and q_flag:
                question.append(" ".join(fu_text[q1: index]))
                q_flag = False
            if fu_text[index][:2] == "答案" :
                tmp = index
                # answers.append(fu_text[index].split(":")[-1])
            if fu_text[index] == "】":
                answer = fu_text[tmp+1: index]
                if len(answer) == 1:
                    if answer[0] == "对": answers.append(1)
                    elif answer[0] == "错": answers.append(0)
                    else: answers.extend(answer)
                else:
                    answers.append(fu_text[tmp+1: index])
            # 
    except Exception as e:
        print(e)
    return question, answers

In [ ]:
question = []
answers = []
fu_text = driver.find_elements(By.TAG_NAME, "font")[1].text.split("\n")
fu_text = [x for x in fu_text if x != '']

start = False
atoz = [chr(x).upper() for x in range(65, 91)]
for index in range(len(fu_text)):
    if fu_text[index] != '第一章' and not start:
        continue
    else: start = True

    if fu_text[index].replace('.', '').split("【")[0].isdigit():
        q_type = fu_text[index].split("】")[0].strip("【")
        q = fu_text[index].split("】")[-1]
        if len(q) == 0:
            q = fu_text[index+1]
            tmp = index+1
        else: tmp = index
        question.append(q)
        alphas = []
        answer = []
        for alpha in q:
            if alpha in atoz:
                alphas.append(alpha)
        for alpha in alphas:
            if alpha == "A": answer.append(fu_text[tmp+1])
            elif alpha == "B": answer.append(fu_text[tmp+2])
            elif alpha == "C": answer.append(fu_text[tmp+3])
            elif alpha == "D": answer.append(fu_text[tmp+4])
            elif alpha == "E": answer.append(fu_text[tmp+5])
            elif alpha == "F": answer.append(fu_text[tmp+6])
        if len(answer) == 1:
            answers.extend(answer)
        else:
            answers.append(answer)

In [69]:
from main import *

option = wb.EdgeOptions()
driver = wb.Edge(options=option)
driver.get('http://www.iamooc.com/2000/053183.htm')

In [70]:
question = []
answers = []
fu_text = driver.find_elements(By.TAG_NAME, "font")[1].text.split("\n")
fu_text = [x for x in fu_text if x != '']

In [71]:
start = False
atoz = [chr(x).upper() for x in range(65, 91)]
for index in range(len(fu_text)):
    if fu_text[index] != '第一章 单元测试' and not start:
        continue
    else: start = True
    line = fu_text[index].split("、")[0]
    if line.isdigit() and start:
        question.append(fu_text[index])
    if fu_text[index][: 2] == "答案":
        tmp = index + 1
    if fu_text[index] == '】':
        true_answer = fu_text[tmp: index]
        if len(true_answer) == 1:
            answers.extend(true_answer)
        else: answers.append(true_answer)

In [74]:
a = answers.copy()

In [75]:
question, answers2 = getQuestionAnswer(driver, "http://www.iamooc.com/2000/053183.htm")

In [77]:
answers2 = [x for x in answers2 if x != []]

In [79]:
answers = a + answers2

In [81]:
to_json(question, answers, title="名画中的瘟疫史")
